# Fine-tuning Embeddings for RAG on Specific Data

As we start our "fine-tuning" week, we'll start with the lowest hanging improvement one can do for RAG - which is:

Fine-tuning embeddings!

- 🤝 Breakout Room #1:
  - Task 1: Dependencies and Boilerplate
  - Task 2: Loading Data
  - Task 3: Constructing a Fine-tuning Dataset
  - Task 4: Fine-tuning `snowflake-arctic-embed-m`
  - Task 5: Evaluating our Retriever

- 🤝 Breakout Room #2:
  - Task 1: Vibe Checking Our LCEL RAG Chain
  - Task 2: Ragas Evaluation



#### Basic Overview of Fine-tuning Embeddings

In essence, what we want to do when we fine-tune our embedding models is very simple:

```
Move the embeddings for questions relating to a document
closer together with that document
```

We can think of fine-tuning our embedding models as follows:

1) We have some pair of text items that *should* be closer together
  - `Question`, `Document` pairs
  - EX: `Who drives the bus?`, `The bus was driven by Kyle, the Bus Driver`.

2) We use these pairs as labeled data to fine-tune our embedding model.

The process of training helps the model more accurately associate our questions with the correct documents.

##### ❓ Question #1:

Describe the nuance between using Q&D pairs to train the embedding model vs. inter-document pairs/related sentences.

What caveats does this approach have? Are there any special considerations for what kind of Q's we should use?

---

**ANSWER:**

We are specifically relating *the questions* to *the documents*. This means that we are making our embedding model at the very specific task of relating potential questions to specific documents.

There are many caveats, but the main ones are:

- Your Q's should reflect the Q's of your users
- This kind of fine-tuning will (purposefully) "overfit" on your data; this is the desired result in this case.

## Task 1: Dependencies and Boilerplate

We'll set up our `nest_asyncio` so we can leverage async loops in our Notebook.

We'll also install the required libraries we'll be using today, and set up our OpenAI API key!

### Nest Asyncio

In [6]:
import nest_asyncio

nest_asyncio.apply()

### Install Dependencies

In [ ]:
!pip install -qU langchain_openai langchain_huggingface langchain_core langchain langchain_community langchain-text-splitters

In [ ]:
!pip install -qU faiss-cpu unstructured==0.15.7 python-pptx==1.0.2 nltk==3.9.1

### Provide OpenAI API Key

In [7]:
import os
# import getpass

# os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter Your OpenAI API Key: ")

## Task 2: Loading Data

We'll be using a recent document released by the EU 'laying down harmonised rules on artificial intelligence and amending Regulations'.

The data can be found [here](https://eur-lex.europa.eu/legal-content/EN/TXT/?uri=CELEX%3A32024R1689), though we will be using a HTML version which was collected into the AIM DataRepository.

First, we'll clone and then `cd` into the DataRepository.

In [ ]:
!git clone https://github.com/AI-Maker-Space/DataRepository.git

In [ ]:
%cd DataRepository

Next we're going to be using the `UnstructuredHTMLLoader` to load our HTML document into a LangChain document using the [Unstructured](https://api.python.langchain.com/en/latest/document_loaders/langchain_community.document_loaders.html.UnstructuredHTMLLoader.html) library.

In [9]:
from langchain_community.document_loaders import UnstructuredHTMLLoader
from langchain_community.document_loaders import BSHTMLLoader

training_documents_loaded = UnstructuredHTMLLoader("DataRepository/eu_ai_act.html") # Does not work as tries to access internet address which is blocked by firewall
# training_documents_loaded = BSHTMLLoader("DataRepository/eu_ai_act.html", open_encoding="utf-8")

Next, we'll set up a classic naive chunking strategy as we only care that the documents get parsed into chunks that we can generate synthetic questions about.

In [10]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 750,
    chunk_overlap  = 20,
    length_function = len
)

Next we can load/split these documents as follows.

In [11]:
docs_loaded = training_documents_loaded.load()

In [12]:
training_documents = text_splitter.split_documents(training_documents_loaded.load())

In [13]:
len(training_documents)

1029

Next, we're going to associate each of our chunks with a unique identifier.

In [14]:
import uuid

id_set = set()

for document in training_documents:
  id = str(uuid.uuid4())
  while id in id_set:
    id = uuid.uuid4()
  id_set.add(id)
  document.metadata["id"] = id

Next, we'll simply use naive Python slicing to create a training, test, and validation set to prepare our data for the next step.

In [15]:
training_split_documents = training_documents[:300]
val_split_documents = training_documents[300:350]
test_split_documents = training_documents[350:400]

## Task 3: Constructing a Fine-tuning Dataset

Using the nodes we created above, we can finally start constructing a fine-tuning dataset utilizing OpenAI's `gpt-4o-mini` (released [July 18th](https://openai.com/index/gpt-4o-mini-advancing-cost-efficient-intelligence/)).

The basic idea here is straightforward enough:

1. We look at a document
2. We generate questions that could be answered by that node

This gives us a number of question/context pairs that we can use to fine-tune our Embeddings model.

In [1]:
# from langchain_openai import ChatOpenAI
from pydantic import BaseModel
from langchain_openai import AzureChatOpenAI

# qa_chat_model = ChatOpenAI(
#     model="gpt-4o-mini",
#     temperature=0
# )

qa_chat_model = AzureChatOpenAI(
    azure_deployment="gpt-4o-mini",
    temperature=0
)

We'll create a simple Question Generation prompt to query `gpt-4o-mini` to generate Questions for each retrieved context.

In [2]:
from langchain_core.prompts import ChatPromptTemplate

qa_prompt = """\
Given the following context, you must generate questions based on only the provided context.

You are to generate {n_questions} questions which should be provided in the following format:

1. QUESTION #1
2. QUESTION #2
...

Context:
{context}
"""

qa_prompt_template = ChatPromptTemplate.from_template(qa_prompt)

We'll create a simple chain to query the LLM!

In [3]:
question_generation_chain = qa_prompt_template | qa_chat_model

There's a lot going on in this function - let's take a deeper look:

1. First, we provide a list of documents and a number of questions
2. We, for each document in our list, generate `n_questions` of questions.
3. We then associate those questions and contexts via a `UUID`.

> NOTE: The reason we're doing this `UUID` association is for ease of use later in the notebook.

##### 🏗️ Activity #1:

We have:

- Lists of `Documents` with the `metadata` field `id`.

We need:

- An object with key `id`, which have values `str` questions.
- An object with key `question_id`, which have values `List(str)` which will be a list of associated `context_id`.

An Example:

question_object:
```python
{
'b4b95fb6-f827-4454-aa5b-20e62733f172': 'What types of accessible formats are available for persons with disabilities?',
'df58ee4f-714c-419e-8324-94e5870574e2': 'How do accessible formats benefit persons with disabilities?',
'505fce8b-0e56-48de-a251-61027e396918': 'What are some of the risks associated with the increasing capabilities of AI systems that generate synthetic content?',
'8ff0ab33-60dc-4fee-8958-91bfb686aca8': 'Why is it important for providers of AI systems to embed technical solutions for marking and detecting synthetic content?'
}
 ```

 context_object:
 ```python
{
'b4b95fb6-f827-4454-aa5b-20e62733f172': ['dd75bf94-75f3-4603-8e4b-5522f6925638'],
'df58ee4f-714c-419e-8324-94e5870574e2': ['dd75bf94-75f3-4603-8e4b-5522f6925638'],
'505fce8b-0e56-48de-a251-61027e396918': ['ffe3893f-688c-48e8-90bd-7a9feb953d90'],
'8ff0ab33-60dc-4fee-8958-91bfb686aca8': ['ffe3893f-688c-48e8-90bd-7a9feb953d90'],
}
 ```

 As you can see, a piece of context can be associated with more than 1 question.

 The task is to write the Python function(s) to accomplish this task.

 Your function signature is provided below, along with the desired return values.

 > NOTE: You can make any modifications that you desire - assuming that you have the correct input and outputs.

In [4]:
import tqdm
def create_questions(documents, n_questions):
    questions: dict[str, Any] = {}
    relevant_docs: dict[str, Any] = {}
    for document in tqdm.tqdm(documents):
        questions_generated = question_generation_chain.invoke({"context": document.page_content, "n_questions": n_questions})
        for question in questions_generated.content.split("\n"):
            question_id = str(uuid.uuid4())
            questions[question_id] = "".join(question.split(".")[1:]).strip()
            relevant_docs[question_id] = [document.metadata["id"]]
    return questions, relevant_docs

We'll use the function to generate training, validation, and test data with `n_questions=2` for each.

In [92]:
training_questions, training_relevant_contexts = create_questions(training_split_documents, 2)

  0%|          | 0/300 [00:00<?, ?it/s]

100%|██████████| 300/300 [04:38<00:00,  1.08it/s]


In [16]:
training_questions, training_relevant_contexts = create_questions(training_split_documents[:3], 2)

100%|██████████| 3/3 [00:03<00:00,  1.04s/it]


In [17]:
training_questions

{'9c4b872c-ea28-49a5-82ed-cd61c09c7b11': 'What is the main purpose of Regulation (EU) 2024/1689 as outlined in the context?',
 'f9f117aa-da31-4951-9f3b-5037c3b2dd42': 'Which specific regulations and directives are amended by the Artificial Intelligence Act mentioned in the context?',
 '57ae3716-d9e4-4e1d-8004-a545fbf960d6': 'What opinions were considered after the transmission of the draft legislative act to the national parliaments?',
 'c010e423-92e4-4ce0-b21c-fda60580aa4b': 'Which legislative procedure is being followed in the context of the draft legislative act?',
 '7158bf62-a99d-468e-9dce-aebae21a47b7': 'What is the primary purpose of the Regulation regarding artificial intelligence systems in the Union?',
 '4f0d4ef2-e4b1-4dff-95ad-dfbe9596c8fc': 'How does the Regulation aim to ensure the protection of health, safety, and fundamental rights in relation to AI systems?'}

In [93]:
val_questions, val_relevant_contexts = create_questions(val_split_documents, 2)

100%|██████████| 50/50 [00:48<00:00,  1.03it/s]


In [94]:
test_questions, test_relevant_contexts = create_questions(test_split_documents, 2)

100%|██████████| 50/50 [00:48<00:00,  1.03it/s]


### Reformating and Saving Datasets

Now, we can save our datasets for later use!

> NOTE: If you ran into issues creating the data - you can use the data from the DataRespository. It's simply called: `train_dataset.jsonl`, etc.

In [18]:
training_corpus = {train_item.metadata["id"] : train_item.page_content for train_item in training_split_documents}

In [20]:
training_corpus

{'8709d20c-3396-47bd-b526-1348ce522987': 'Official Journal of the European Union EN L series\n\n2024/1689 12.7.2024\n\nREGULATION (EU) 2024/1689 OF THE EUROPEAN PARLIAMENT AND OF THE COUNCIL\n\nof 13 June 2024\n\nlaying down harmonised rules on artificial intelligence and amending Regulations (EC) No 300/2008, (EU) No 167/2013, (EU) No 168/2013, (EU) 2018/858, (EU) 2018/1139 and (EU) 2019/2144 and Directives 2014/90/EU, (EU) 2016/797 and (EU) 2020/1828 (Artificial Intelligence Act)\n\n(Text with EEA relevance)\n\nTHE EUROPEAN PARLIAMENT AND THE COUNCIL OF THE EUROPEAN UNION,\n\nHaving regard to the Treaty on the Functioning of the European Union, and in particular Articles 16 and 114 thereof,\n\nHaving regard to the proposal from the European Commission,',
 '36d0fb72-9207-43c0-aef8-24bbae9e1724': 'After transmission of the draft legislative act to the national parliaments,\n\nHaving regard to the opinion of the European Economic and Social Committee (1),\n\nHaving regard to the opinion

In [95]:
import json

training_corpus = {train_item.metadata["id"] : train_item.page_content for train_item in training_split_documents}

train_dataset = {
    "questions" : training_questions,
    "relevant_contexts" : training_relevant_contexts,
    "corpus" : training_corpus
}

with open("DataRepository/training_dataset.jsonl", "w") as f:
  json.dump(train_dataset, f)

In [96]:
val_corpus = {val_item.metadata["id"] : val_item.page_content for val_item in val_split_documents}

val_dataset = {
    "questions" : val_questions,
    "relevant_contexts" : val_relevant_contexts,
    "corpus" : val_corpus
}

with open("DataRepository/val_dataset.jsonl", "w") as f:
  json.dump(val_dataset, f)

In [97]:
train_corpus = {test_item.metadata["id"] : test_item.page_content for test_item in test_split_documents}

test_dataset = {
    "questions" : test_questions,
    "relevant_contexts" : test_relevant_contexts,
    "corpus" : train_corpus
}

with open("DataRepository/test_dataset.jsonl", "w") as f:
  json.dump(test_dataset, f)

## Task 4: Fine-tuning `snowflake-arctic-embed-m`

Now that we have a dataset, let's grab a `sentence-transformers` Embeddings model!

We'll be using Snowflake's [`snowflake-arctic-embed-m`](https://huggingface.co/Snowflake/snowflake-arctic-embed-m) as a base embeddings model.

It is a well performing embeddings model by itself, but there's a lot of very specific domain terms and vocabulary in our courpus - so lets fine-tune it and see what that can do for us!

In [20]:
!pip install -qU sentence_transformers datasets pyarrow

In [98]:
import torch

if torch.cuda.is_available():
    print(f"CUDA is available. Device name: {torch.cuda.get_device_name(0)}")
else:
    print("CUDA is not available.")

CUDA is available. Device name: NVIDIA RTX 2000 Ada Generation Laptop GPU


In [99]:
from sentence_transformers import SentenceTransformer

model_id = "Snowflake/snowflake-arctic-embed-m"
model = SentenceTransformer(model_id).to("cuda")

We'll grab some necessary imports from `sentence_transformers` and `torch`.

> NOTE: PyTorch (`torch`) is a popular machine learning library - while we don't go very deep into PyTorch it's an incredibly powerful and interesting library! Please read more about it [here](https://pytorch.org/tutorials/beginner/basics/intro.html)!

In [100]:
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from sentence_transformers import InputExample

We're using a toy batch size here to reflect the limited number of examples we have.

> NOTE: It is typical to use a much larger batch size (~64+), hardware permitting.

In [101]:
BATCH_SIZE = 20

Let's move our dataset into the expected format for training.

In [102]:
import json
# Load the saved training dataset
with open("DataRepository/training_dataset.jsonl", "r") as f:
    train_dataset = json.load(f)

In [103]:
corpus = train_dataset['corpus']
queries = train_dataset['questions']
relevant_docs = train_dataset['relevant_contexts']

examples = []
for query_id, query in queries.items():
    doc_id = relevant_docs[query_id][0]
    text = corpus[doc_id]
    example = InputExample(texts=[query, text])
    examples.append(example)

Now we can create a `torch` `DataLoader`!

In [104]:
loader = DataLoader(
    examples, batch_size=BATCH_SIZE
)

Next up, we'll prepare our loss function!

Loss is an important part of training, fine-tuning, and more. If you want a deep dive on loss - you can check out our [event on loss!](https://www.youtube.com/watch?v=iB8FWR9aD5Q&t=8s).

The core loss we're using today is called `MultipleNegativesRankingLoss` - you can find more information [here](https://github.com/UKPLab/sentence-transformers/blob/master/sentence_transformers/losses/MultipleNegativesRankingLoss.py).

This is "wrapped" in `MatryoshkaLoss`, which you can read the implementation of [here](https://github.com/UKPLab/sentence-transformers/blob/master/sentence_transformers/losses/MatryoshkaLoss.py).

In [105]:
from sentence_transformers.losses import MatryoshkaLoss, MultipleNegativesRankingLoss

matryoshka_dimensions = [768, 512, 256, 128, 64]
inner_train_loss = MultipleNegativesRankingLoss(model)
train_loss = MatryoshkaLoss(
    model, inner_train_loss, matryoshka_dims=matryoshka_dimensions
)

##### 🏗️ Activity #2:

Both of these losses sound "cool", but what are they - exactly - under the hood?

Why are these losses specifically doing? Please write a short summary of each loss.

> NOTE: This is a course focused on AI Engineering and the application of AI - looking for a hint? Try pasting the code (linked above) into ChatGPT/Claude to write the summary!

#### ✔️ Answer #2:

**Multiple Negative Ranking Loss (MNRL)** is a machine learning technique designed to enhance model training by contrasting positive samples with multiple negative samples, improving differentiation between relevant and irrelevant data. It minimizes the distance between a query and positive samples while maximizing it with multiple negative ones, offering a more nuanced learning process, particularly in complex scenarios like recommendation systems, ad targeting, and information retrieval. MNRL boosts discriminative power, generalization, and scalability, but its challenges include complexity in selecting negative samples and increased computational demands. Its applications span fields like natural language processing, search engine optimization, and user engagement prediction.

Reference: [Multiple Negative Ranking Loss (MNRL) Explained](https://medium.com/@aisagescribe/multiple-negative-ranking-loss-mnrl-explained-5b4741e38d8f)

**Matryoshka Loss** is inspired by Matryoshka dolls (nested dolls of decreasing size) and is designed to work with embeddings of varying dimensions. It's structured to optimize embeddings at multiple levels of granularity (higher and lower-dimensional embeddings), which can help improve generalization.

- Mechanism: It works by optimizing the embeddings to maintain consistency across different dimensions. For instance, it ensures that a higher-dimensional embedding (e.g., 768-dim) contains more fine-grained information while a lower-dimensional embedding (e.g., 64-dim) is more abstract but consistent with the higher-dimensional embedding.
- Objective: The loss ensures that embeddings at all levels (from high-dimensional to low-dimensional) preserve a consistent relational structure. This enables the model to compress information without losing important relational properties.

##### Why Use These Losses Together?

The Multiple Negatives Ranking Loss helps to optimize the quality of embeddings by contrasting positive and negative examples, while the Matryoshka Loss ensures that embeddings of different dimensionalities remain consistent. When combined, they help train a model that produces both effective and scalable embeddings across different dimensions.

Now we can set-up our evaluator.

> NOTE: Due to the formatting of our dataset - this is all we have to do!

In [106]:
# Load the saved validation dataset
with open("DataRepository/val_dataset.jsonl", "r") as f:
    val_dataset = json.load(f)

In [107]:
from sentence_transformers.evaluation import InformationRetrievalEvaluator

corpus = val_dataset['corpus']
queries = val_dataset['questions']
relevant_docs = val_dataset['relevant_contexts']

evaluator = InformationRetrievalEvaluator(queries, corpus, relevant_docs)

We'll train this model for 5 epochs, though you could increase this number if we had a significant amount more data.

In [108]:
EPOCHS = 5

It's training time!

> NOTE: We're manually defining a warm-up period here - this is just to provide a smooth ramp into our training!

In [109]:
warmup_steps = int(len(loader) * EPOCHS * 0.1)

model.fit(
    train_objectives=[(loader, train_loss)],
    epochs=EPOCHS,
    warmup_steps=warmup_steps,
    output_path='finetuned_arctic',
    show_progress_bar=True,
    evaluator=evaluator,
    evaluation_steps=50,
)

 33%|███▎      | 50/150 [01:14<02:21,  1.41s/it]

{'eval_cosine_accuracy@1': 0.89, 'eval_cosine_accuracy@3': 0.97, 'eval_cosine_accuracy@5': 0.99, 'eval_cosine_accuracy@10': 1.0, 'eval_cosine_precision@1': 0.89, 'eval_cosine_precision@3': 0.3233333333333333, 'eval_cosine_precision@5': 0.19799999999999998, 'eval_cosine_precision@10': 0.09999999999999998, 'eval_cosine_recall@1': 0.89, 'eval_cosine_recall@3': 0.97, 'eval_cosine_recall@5': 0.99, 'eval_cosine_recall@10': 1.0, 'eval_cosine_ndcg@10': 0.9487220907111209, 'eval_cosine_mrr@10': 0.9316666666666668, 'eval_cosine_map@100': 0.9316666666666665, 'eval_dot_accuracy@1': 0.89, 'eval_dot_accuracy@3': 0.97, 'eval_dot_accuracy@5': 0.99, 'eval_dot_accuracy@10': 1.0, 'eval_dot_precision@1': 0.89, 'eval_dot_precision@3': 0.3233333333333333, 'eval_dot_precision@5': 0.19799999999999998, 'eval_dot_precision@10': 0.09999999999999998, 'eval_dot_recall@1': 0.89, 'eval_dot_recall@3': 0.97, 'eval_dot_recall@5': 0.99, 'eval_dot_recall@10': 1.0, 'eval_dot_ndcg@10': 0.9487220907111209, 'eval_dot_mrr@10'

 67%|██████▋   | 100/150 [02:41<01:21,  1.63s/it]

{'eval_cosine_accuracy@1': 0.88, 'eval_cosine_accuracy@3': 0.97, 'eval_cosine_accuracy@5': 0.99, 'eval_cosine_accuracy@10': 1.0, 'eval_cosine_precision@1': 0.88, 'eval_cosine_precision@3': 0.3233333333333333, 'eval_cosine_precision@5': 0.19799999999999998, 'eval_cosine_precision@10': 0.09999999999999998, 'eval_cosine_recall@1': 0.88, 'eval_cosine_recall@3': 0.97, 'eval_cosine_recall@5': 0.99, 'eval_cosine_recall@10': 1.0, 'eval_cosine_ndcg@10': 0.9448026497090888, 'eval_cosine_mrr@10': 0.9264285714285713, 'eval_cosine_map@100': 0.9264285714285714, 'eval_dot_accuracy@1': 0.88, 'eval_dot_accuracy@3': 0.97, 'eval_dot_accuracy@5': 0.99, 'eval_dot_accuracy@10': 1.0, 'eval_dot_precision@1': 0.88, 'eval_dot_precision@3': 0.3233333333333333, 'eval_dot_precision@5': 0.19799999999999998, 'eval_dot_precision@10': 0.09999999999999998, 'eval_dot_recall@1': 0.88, 'eval_dot_recall@3': 0.97, 'eval_dot_recall@5': 0.99, 'eval_dot_recall@10': 1.0, 'eval_dot_ndcg@10': 0.9448026497090888, 'eval_dot_mrr@10'

100%|██████████| 150/150 [03:55<00:00,  1.36s/it]

{'eval_cosine_accuracy@1': 0.88, 'eval_cosine_accuracy@3': 0.98, 'eval_cosine_accuracy@5': 0.99, 'eval_cosine_accuracy@10': 1.0, 'eval_cosine_precision@1': 0.88, 'eval_cosine_precision@3': 0.3266666666666667, 'eval_cosine_precision@5': 0.19799999999999998, 'eval_cosine_precision@10': 0.09999999999999998, 'eval_cosine_recall@1': 0.88, 'eval_cosine_recall@3': 0.98, 'eval_cosine_recall@5': 0.99, 'eval_cosine_recall@10': 1.0, 'eval_cosine_ndcg@10': 0.9468051816640692, 'eval_cosine_mrr@10': 0.9289285714285712, 'eval_cosine_map@100': 0.9289285714285714, 'eval_dot_accuracy@1': 0.88, 'eval_dot_accuracy@3': 0.98, 'eval_dot_accuracy@5': 0.99, 'eval_dot_accuracy@10': 1.0, 'eval_dot_precision@1': 0.88, 'eval_dot_precision@3': 0.3266666666666667, 'eval_dot_precision@5': 0.19799999999999998, 'eval_dot_precision@10': 0.09999999999999998, 'eval_dot_recall@1': 0.88, 'eval_dot_recall@3': 0.98, 'eval_dot_recall@5': 0.99, 'eval_dot_recall@10': 1.0, 'eval_dot_ndcg@10': 0.9468051816640692, 'eval_dot_mrr@10'

100%|██████████| 150/150 [03:56<00:00,  1.57s/it]

{'train_runtime': 236.2049, 'train_samples_per_second': 12.701, 'train_steps_per_second': 0.635, 'train_loss': 1.2252303059895833, 'epoch': 5.0}


## Task 5: Evaluating our Retriever

Now that we have fine-tuned our retriever - let's see if it's worthwhile!

We'll start with some basic imports.

In [110]:
import pandas as pd

from langchain_community.vectorstores import FAISS
# from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_openai.embeddings import AzureOpenAIEmbeddings
from langchain_core.documents import Document

Now we'll define a function that will help us evaluate our retrieval process.

> NOTE: We're assuming 1 correct document in a "hit".

In [111]:
def evaluate_openai(
    dataset,
    embed_model,
    top_k=5,
    verbose=False,
):
  corpus = dataset['corpus']
  questions = dataset['questions']
  relevant_docs = dataset['relevant_contexts']
  documents = [Document(page_content=content, metadata={"id": doc_id}) for doc_id, content in corpus.items()]
  vectorstore = FAISS.from_documents(documents, embed_model)

  retriever = vectorstore.as_retriever(search_kwargs={"k": top_k})

  eval_results = []
  for id, question in tqdm(questions.items()):
    retrieved_nodes = retriever.invoke(question)
    retrieved_ids = [node.metadata["id"] for node in retrieved_nodes]
    expected_id = relevant_docs[id][0]
    is_hit = expected_id in retrieved_ids
    eval_results.append({"id": id, "question": question, "expected_id": expected_id, "is_hit": is_hit})

  return eval_results

All that's left to do is evaluate, we'll evaluate our model against:

1. OpenAI's closed source `text-embedding-3-small`
2. The base non-fine-tuned version of `Snowflake/snowflake-arctic-embed-m`.

Let's see how it stacks up!

### `text-embedding-3-small`

### `text-embedding-3-large`

We are going to use **large** model instead

In [112]:
# Load the saved test dataset
with open("DataRepository/test_dataset.jsonl", "r") as f:
    test_dataset = json.load(f)

In [113]:
from tqdm import tqdm
# te3_openai = OpenAIEmbeddings(model="text-embedding-3-small")
te3_openai = AzureOpenAIEmbeddings(model="text-embedding-3-large")
te3_results = evaluate_openai(test_dataset, te3_openai)

100%|██████████| 100/100 [00:41<00:00,  2.42it/s]


In [114]:
te3_results_df = pd.DataFrame(te3_results)

In [115]:
te3_hit_rate = te3_results_df["is_hit"].mean()
te3_hit_rate

0.98

### `Snowflake/snowflake-arctic-embed-m` (base)

In [116]:
from langchain_huggingface import HuggingFaceEmbeddings

huggingface_embeddings = HuggingFaceEmbeddings(model_name="Snowflake/snowflake-arctic-embed-m")
arctic_embed_m_results = evaluate_openai(test_dataset, huggingface_embeddings)

100%|██████████| 100/100 [00:03<00:00, 33.17it/s]


In [117]:
arctic_embed_m_results_df = pd.DataFrame(arctic_embed_m_results)

In [118]:
arctic_embed_m_hit_rate = arctic_embed_m_results_df["is_hit"].mean()
arctic_embed_m_hit_rate

0.61

### `Snowflake/snowflake-arctic-embed-m` (fine-tuned)

In [119]:
finetune_embeddings = HuggingFaceEmbeddings(model_name="finetuned_arctic")
finetune_results = evaluate_openai(test_dataset, finetune_embeddings)

Some weights of BertModel were not initialized from the model checkpoint at finetuned_arctic and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 100/100 [00:03<00:00, 32.40it/s]


In [120]:
finetune_results_df = pd.DataFrame(finetune_results)

In [121]:
finetune_hit_rate = finetune_results_df["is_hit"].mean()
finetune_hit_rate

0.99

# 🤝 Breakout Room #2

## Task 1: Vibe Checking the RAG Pipeline

We're going to use our RAG pipeline to vibe check on some common phrases now that we've modified it!

### Creating New Chunks

In order to try and evaluate our system more fairly, let's create new chunks that we will use to create our Vector Store.

In [122]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 600,
    chunk_overlap  = 50,
    length_function = len
)

training_documents = text_splitter.split_documents(training_documents_loaded.load())

### Base Chain

We'll start by constructing our base chain, which will use the untrained retrieval model.

#### R - Retrieval

In [123]:
from langchain_community.vectorstores import FAISS

base_vectorstore = FAISS.from_documents(training_documents, huggingface_embeddings)
base_retriever = base_vectorstore.as_retriever(search_kwargs={"k": 6})

#### A - Augmented

In [124]:
from langchain_core.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Given a provided context and a question, you must answer the question. If you do not know the answer, you must state that you do not know.

Context:
{context}

Question:
{question}

Answer:
"""

rag_prompt_template = ChatPromptTemplate.from_template(RAG_PROMPT)

#### G - Generation

In [125]:
rag_llm =  AzureChatOpenAI(
    azure_deployment="gpt-4o-mini",
    temperature=0
)

#### RAG - LCEL RAG Pipeline

In [126]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel

base_rag_chain = (
    {"context": itemgetter("question") | base_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt_template | rag_llm | StrOutputParser(), "context": itemgetter("context")}
)

In [127]:
base_rag_chain.invoke({"question" : "Why does the EU want to regulate AI?"})["response"]

'The EU wants to regulate AI to promote a human-centric approach to AI, ensure the protection of ethical principles, safeguard health and safety, protect fundamental rights, and boost innovation and employment. The regulation aims to make the Union a leader in the development of secure, trustworthy, and ethical AI while facilitating the protection of natural persons, democracy, the rule of law, and environmental protection.'

In [128]:
base_rag_chain.invoke({"question" : "What are the codes of practice?"})["response"]

'I do not know.'

In [129]:
base_rag_chain.invoke({"question" : "How many parameters is too many parameters?"})["response"]

'The context suggests that models with at least a billion parameters are considered to display significant generality. Therefore, it can be inferred that having a billion parameters or more is seen as a threshold for significant capability. However, the context does not specify a maximum limit for parameters. Thus, I do not know how many parameters would be considered "too many."'

In [130]:
base_rag_chain.invoke({"question" : "What is an emotion recognition system and why is it important?"})["response"]

'An emotion recognition system is a type of artificial intelligence (AI) technology designed to identify and interpret human emotions based on various inputs, such as facial expressions, voice tone, and biometric data. These systems analyze patterns to infer emotional states, aiming to understand how individuals feel in different contexts.\n\nThe importance of emotion recognition systems lies in their potential applications across various fields, including mental health, customer service, security, and human-computer interaction. However, there are significant concerns regarding their reliability, specificity, and generalizability, as emotions can vary widely across cultures and individuals. Misinterpretations can lead to discriminatory outcomes and infringe on personal rights and freedoms, highlighting the need for careful consideration and regulation in their development and deployment.'

### Fine-tuned Embedding Model

Now let's rebuild our RAG chain with the Fine-tuned model - the only component we need to change is our `FAISS` vectorstore!

In [131]:
finetune_vectorstore = FAISS.from_documents(training_documents, finetune_embeddings)
finetune_retriever = finetune_vectorstore.as_retriever(search_kwargs={"k": 6})

In [132]:
finetune_rag_chain = (
    {"context": itemgetter("question") | finetune_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt_template | rag_llm | StrOutputParser(), "context": itemgetter("context")}
)

In [133]:
finetune_rag_chain.invoke({"question" : "Why does the EU want to regulate AI?"})["response"]

'The EU wants to regulate AI to improve the functioning of the internal market, promote the uptake of human-centric and trustworthy artificial intelligence, and ensure a high level of protection of health, safety, and fundamental rights. This includes safeguarding democracy, the rule of law, and environmental protection against the harmful effects of AI systems, while also supporting innovation.'

In [134]:
finetune_rag_chain.invoke({"question" : "What are the codes of practice?"})["response"]

'Codes of practice are guidelines developed to ensure compliance with obligations under the AI Regulation for providers of general-purpose AI models. They aim to address systemic risks, establish a risk taxonomy, and provide specific risk assessment and mitigation measures. The AI Office is responsible for encouraging and facilitating the creation, review, and adaptation of these codes, taking into account international approaches and involving various stakeholders. The codes should be ready by 2 May 2025.'

In [135]:
finetune_rag_chain.invoke({"question" : "How many parameters is too many parameters?"})["response"]

'The context suggests that models with at least a billion parameters are considered to display significant generality and can competently perform a wide range of distinctive tasks. Therefore, it implies that having a billion parameters is a threshold for significant generality, but it does not specify a maximum limit for "too many" parameters. Thus, I do not know how many parameters would be considered "too many."'

In [136]:
finetune_rag_chain.invoke({"question" : "What is an emotion recognition system and why is it important?"})["response"]

'An emotion recognition system is defined as an AI system that identifies or infers the emotions or intentions of natural persons based on their biometric data. This includes emotions such as happiness, sadness, anger, surprise, disgust, embarrassment, excitement, shame, contempt, satisfaction, and amusement. \n\nThe importance of emotion recognition systems lies in their potential applications across various fields, such as marketing, security, healthcare, and education. However, there are significant concerns regarding their reliability, specificity, and generalizability, as emotional expressions can vary widely across cultures and individuals. These concerns highlight the risks of discriminatory outcomes and potential intrusions on personal rights and freedoms.'

##### ❓Question #2:

Which LCEL RAG Chain do you think answered the questions better, and why?

#### ✔️ Answer #2:

The fine-tuned model produces better result, as it enables to answer the *"What are the codes of practice?"* question which the regular RAG Chain was not able to answer and produced "I do not know" answer. It has to do with the better retrieval as the fine-tuned model can better match a question with an answer.

## Task 2: RAGAS Evaluation

It's great to have some idea of how our system is doing based on vibe-checks, but let's use RAGAS to provide more insight info. on how things are improving!

In [137]:
!pip install -qU ragas

### RAGAS Synthetic Testset Generation

First things first, we need to generate some data to test our model on.

Let's use our test data that we created before as a base!

In [138]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from langchain_openai import AzureOpenAIEmbeddings

generator_llm = AzureChatOpenAI(model="gpt-4o-mini")
critic_llm = AzureChatOpenAI(model="gpt-4")
embeddings = AzureOpenAIEmbeddings(azure_deployment="text-embedding-3-large")

In [139]:
generator = TestsetGenerator.from_langchain(
    generator_llm,
    critic_llm,
    embeddings
)

In [ ]:
testset = generator.generate_with_langchain_docs(test_split_documents, test_size=20, distributions={simple: 0.5, reasoning: 0.25, multi_context: 0.25})

In [141]:
testset.to_pandas().head()

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,What role does the Council play in the prepara...,"[the preparation of delegated acts, the Europe...",The answer to given question is not present in...,simple,"[{'source': 'DataRepository/eu_ai_act.html', '...",True
1,What provisions are made for downstream provid...,"[the AI Office, it should provide for the poss...",The context mentions that the AI Office should...,simple,"[{'source': 'DataRepository/eu_ai_act.html', '...",True
2,What is the objective of the Regulation regard...,"[(176) Since the objective of this Regulation,...",The objective of the Regulation is to improve ...,simple,"[{'source': 'DataRepository/eu_ai_act.html', '...",True
3,What is the significance of clarifying the res...,[(161) It is necessary to clarify the responsi...,The significance of clarifying the responsibil...,simple,"[{'source': 'DataRepository/eu_ai_act.html', '...",True
4,What rights do affected persons have when a de...,[(171) Affected persons should have the right ...,Affected persons have the right to obtain an e...,simple,"[{'source': 'DataRepository/eu_ai_act.html', '...",True


### Generating Answer Datasets

For each of our pipelines, let's generate answers to these questions!

Once we have our: Questions, Answers, Contexts, Ground Truths we can move on to evaluating our datasets!

In [142]:
from datasets import Dataset

def generate_answers(chain, testset):
  answers = []
  contexts = []
  questions = testset.to_pandas()["question"].values.tolist()
  ground_truths = testset.to_pandas()["ground_truth"].values.tolist()

  for question in tqdm(questions):
    answer = chain.invoke({"question" : question})
    answers.append(answer["response"])
    contexts.append([context.page_content for context in answer["context"]])

  return Dataset.from_dict({
      "question" : questions,
      "answer" : answers,
      "contexts" : contexts,
      "ground_truth" : ground_truths
  })

In [143]:
base_dataset = generate_answers(base_rag_chain, testset)

100%|██████████| 19/19 [00:21<00:00,  1.12s/it]


In [144]:
finetune_dataset = generate_answers(finetune_rag_chain, testset)

100%|██████████| 19/19 [00:34<00:00,  1.80s/it]


### Evaluating Using the Test Set

Now that we have a test set - it's time to evaluate our pipelines with it!

In [145]:
from ragas.metrics import (
    context_recall,
    context_precision,
)

In [146]:
from ragas import evaluate

result = evaluate(
    base_dataset,
    metrics=[
        context_precision,
        context_recall,
    ],
)

Evaluating: 100%|██████████| 38/38 [00:12<00:00,  2.99it/s]


In [147]:
result

{'context_precision': 0.4531, 'context_recall': 0.6310}

In [148]:
result.to_pandas().head()

,question,contexts,answer,ground_truth,context_precision,context_recall
0,What role does the Council play in the prepara...,[access to meetings of Commission expert group...,I do not know.,The answer to given question is not present in...,0.000000,1.0
1,What provisions are made for downstream provid...,"[therein, the mandatory requirements for high-...",Downstream providers are allowed to lodge comp...,The context mentions that the AI Office should...,0.966667,1.0
2,What is the objective of the Regulation regard...,[of the high-risk AI system with the requireme...,The objective of the Regulation is to improve ...,The objective of the Regulation is to improve ...,0.200000,1.0
3,What is the significance of clarifying the res...,[right to obtain an explanation should not app...,The significance of clarifying the responsibil...,The significance of clarifying the responsibil...,0.700000,0.0
4,What rights do affected persons have when a de...,[(e) the extent to which the use of an AI syst...,The context does not provide specific details ...,Affected persons have the right to obtain an e...,1.000000,0.0


In [149]:
result = evaluate(
    finetune_dataset,
    metrics=[
        context_precision,
        context_recall,
    ],
)

Evaluating: 100%|██████████| 38/38 [00:14<00:00,  2.68it/s]


In [150]:
result

{'context_precision': 0.7465, 'context_recall': 0.9693}

In [151]:
result.to_pandas().head()

,question,contexts,answer,ground_truth,context_precision,context_recall
0,What role does the Council play in the prepara...,[access to meetings of Commission expert group...,The Council plays a role in the preparation of...,The answer to given question is not present in...,0.0,1.0
1,What provisions are made for downstream provid...,[2. Downstream providers shall have the right ...,Downstream providers have the right to lodge a...,The context mentions that the AI Office should...,1.0,1.0
2,What is the objective of the Regulation regard...,[(1) The purpose of this Regulation is to impr...,The objective of the Regulation is to improve ...,The objective of the Regulation is to improve ...,1.0,1.0
3,What is the significance of clarifying the res...,[(161) It is necessary to clarify the responsi...,Clarifying the responsibilities and competence...,The significance of clarifying the responsibil...,1.0,1.0
4,What rights do affected persons have when a de...,[(48) The extent of the adverse impact caused ...,Affected persons have the right to obtain clea...,Affected persons have the right to obtain an e...,1.0,1.0


#### 🏗️ Activity #3:

Discuss changes that you'd make to this pipeline based on the performance improvements that you see with RAGAS and the fine-tuning.

Come up with 3 changes, and then we'll discuss these options as a group!

1. For area specific datasets, I would use subject matter expert to create a set of questions the same way we used LLM. This way it allows to get questions generated by experts and real people.
2. Then we could use those examples to prime (multi-shot prompt) LLM to create a bigger set of questions using human generated examples. 
3. Create a test set generated by humans/SMEs.
4. Add reranking and pre-filtering of chunks by using smaller models like GPT-4o-mini.
5. (Multi-)agent advanced RAG pipelines